DECISION TREE USING ENTROPY

In [7]:
import numpy as np
from collections import Counter
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [8]:
dataset = pd.read_csv('advertising.csv')
dataset.head()

Daily Time Spent on Site  Age  Area Income  Daily Internet Usage  \
0                     68.95   35     61833.90                256.09   
1                     80.23   31     68441.85                193.77   
2                     69.47   26     59785.94                236.50   
3                     74.15   29     54806.18                245.89   
4                     68.37   35     73889.99                225.58   

                           Ad Topic Line            City  Male     Country  \
0     Cloned 5thgeneration orchestration     Wrightburgh     0     Tunisia   
1     Monitored national standardization       West Jodi     1       Nauru   
2       Organic bottom-line service-desk        Davidton     0  San Marino   
3  Triple-buffered reciprocal time-frame  West Terrifurt     1       Italy   
4          Robust logistical utilization    South Manuel     0     Iceland   

             Timestamp  Clicked on Ad  
0  2016-03-27 00:53:11              0  
1  2016-04-04 01:39:02              0  
2  2016-03-13 20:35:42              0  
3  2016-01-10 02:31:19              0  
4  2016-06-03 03:36:18              0

In [9]:
def entropy(y):
    counts = np.bincount(y)
    probabilities = counts / len(y)
    return -np.sum([p * np.log2(p) for p in probabilities if p > 0])

In [10]:
def split_dataset(X, y, feature, threshold):
    left_mask = X[:, feature] <= threshold
    right_mask = ~left_mask
    return X[left_mask], y[left_mask], X[right_mask], y[right_mask]

In [11]:
def best_split(X, y, criterion='entropy'):
    best_gain = 0
    best_feature, best_threshold = None, None
    current_impurity = entropy(y) if criterion == 'entropy' else gini(y)

    for feature in range(X.shape[1]):
        thresholds = np.unique(X[:, feature])
        for threshold in thresholds:
            X_left, y_left, X_right, y_right = split_dataset(X, y, feature, threshold)
            if len(y_left) == 0 or len(y_right) == 0:
                continue

            left_impurity = entropy(y_left) if criterion == 'entropy' else gini(y_left)
            right_impurity = entropy(y_right) if criterion == 'entropy' else gini(y_right)

            weighted_impurity = (len(y_left) / len(y) * left_impurity) + (len(y_right) / len(y) * right_impurity)
            gain = current_impurity - weighted_impurity

            if gain > best_gain:
                best_gain = gain
                best_feature = feature
                best_threshold = threshold

    return best_feature, best_threshold

In [12]:
def build_tree(X, y, depth=0, max_depth=5, criterion='entropy'):
    if len(set(y)) == 1 or depth >= max_depth:
        return Counter(y).most_common(1)[0][0]

    feature, threshold = best_split(X, y, criterion)
    if feature is None:
        return Counter(y).most_common(1)[0][0]

    X_left, y_left, X_right, y_right = split_dataset(X, y, feature, threshold)

    left_subtree = build_tree(X_left, y_left, depth+1, max_depth, criterion)
    right_subtree = build_tree(X_right, y_right, depth+1, max_depth, criterion)

    return {"feature": feature, "threshold": threshold, "left": left_subtree, "right": right_subtree}

In [13]:
def predict(tree, sample):
    if isinstance(tree, dict):
        feature, threshold = tree["feature"], tree["threshold"]
        if sample[feature] <= threshold:
            return predict(tree["left"], sample)
        else:
            return predict(tree["right"], sample)
    else:
        return tree

In [14]:
X_train_np, y_train_np = X_train.to_numpy(), y_train.to_numpy()
X_test_np, y_test_np = X_test.to_numpy(), y_test.to_numpy()

tree_entropy = build_tree(X_train_np, y_train_np, criterion='entropy')

y_pred_entropy = [predict(tree_entropy, sample) for sample in X_test_np]

In [15]:
print("Entropy Decision Tree Performance:")
print(confusion_matrix(y_test_np, y_pred_entropy))
print(classification_report(y_test_np, y_pred_entropy))

Entropy Decision Tree Performance:
[[ 83   6]
 [ 10 101]]
              precision    recall  f1-score   support

           0       0.89      0.93      0.91        89
           1       0.94      0.91      0.93       111

    accuracy                           0.92       200
   macro avg       0.92      0.92      0.92       200
weighted avg       0.92      0.92      0.92       200



DECISION TREE USING GINI

In [16]:
def gini(y):
    counts = np.bincount(y)
    probabilities = counts / len(y)
    return 1 - np.sum([p**2 for p in probabilities])

In [17]:
tree_gini = build_tree(X_train_np, y_train_np, criterion='gini')

y_pred_gini = [predict(tree_gini, sample) for sample in X_test_np]

In [18]:
print("\nGini Decision Tree Performance:")
print(confusion_matrix(y_test_np, y_pred_gini))
print(classification_report(y_test_np, y_pred_gini))


Gini Decision Tree Performance:
[[ 83   6]
 [  9 102]]
              precision    recall  f1-score   support

           0       0.90      0.93      0.92        89
           1       0.94      0.92      0.93       111

    accuracy                           0.93       200
   macro avg       0.92      0.93      0.92       200
weighted avg       0.93      0.93      0.93       200



DECISION TREE USING SCIKIT LEARN

In [19]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(criterion='entropy', max_depth=5, random_state=42)
clf.fit(X_train, y_train)
y_pred_sklearn = clf.predict(X_test)

print("\nScikit-Learn Decision Tree Performance:")
print(confusion_matrix(y_test, y_pred_sklearn))
print(classification_report(y_test, y_pred_sklearn))



Scikit-Learn Decision Tree Performance:
[[ 81   8]
 [  9 102]]
              precision    recall  f1-score   support

           0       0.90      0.91      0.91        89
           1       0.93      0.92      0.92       111

    accuracy                           0.92       200
   macro avg       0.91      0.91      0.91       200
weighted avg       0.92      0.92      0.92       200

